<a href="https://colab.research.google.com/github/raisinghanii/Movie-Recommendation/blob/master/Copy_of_movie_recommendation_using_KNN_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy

In [ ]:
# data science imports
import math
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# utils import
from fuzzywuzzy import fuzz

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# !curl http://files.grouplens.org/datasets/movielens/ml-latest.zip -o movie.zip
# !unzip movie.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  264M  100  264M    0     0   115M      0  0:00:02  0:00:02 --:--:--  115M
Archive:  movie.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


## 1. Load Data

In [ ]:
df_movies = pd.read_csv('ml-latest/movies.csv',usecols=['movieId', 'title'],dtype={'movieId': 'int32', 'title': 'str'})

df_ratings = pd.read_csv('ml-latest/ratings.csv',usecols=['userId', 'movieId', 'rating'],dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
df_movies.head()

movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)

In [ ]:
df_ratings.head()

userId  movieId  rating
0       1      307     3.5
1       1      481     3.5
2       1     1091     1.5
3       1     1257     4.5
4       1     1449     4.5

In [ ]:
num_users = len(df_ratings.userId.unique())
num_items = len(df_ratings.movieId.unique())
print('There are {} unique users and {} unique movies in this data set'.format(num_users, num_items))

There are 283228 unique users and 53889 unique movies in this data set


#### 1. Plot the counts of each rating

we first need to get the counts of each rating from ratings data

In [ ]:
# count of each rating
df_ratings_cnt_tmp = pd.DataFrame(df_ratings.groupby('rating').size(), columns=['count'])
df_ratings_cnt_tmp

count
rating         
0.5      442388
1.0      886233
1.5      441354
2.0     1850627
2.5     1373419
3.0     5515668
3.5     3404360
4.0     7394710
4.5     2373550
5.0     4071135

We can see that above table does not include counts of zero rating score. So we need to add that in rating count dataframe as well

In [ ]:
# there are a lot more counts in rating of zero
total_cnt = num_users * num_items
rating_zero_cnt = total_cnt - df_ratings.shape[0]
# append counts of zero rating to df_ratings_cnt
df_ratings_cnt = df_ratings_cnt_tmp.append(
    pd.DataFrame({'count': rating_zero_cnt}, index=[0.0]),
    verify_integrity=True,
).sort_index()
df_ratings_cnt

count
0.0  15235120248
0.5       442388
1.0       886233
1.5       441354
2.0      1850627
2.5      1373419
3.0      5515668
3.5      3404360
4.0      7394710
4.5      2373550
5.0      4071135

The count for zero rating score is too big to compare with others. So let's take log transform for count values and then we can plot them to compare

In [ ]:
# add log count
df_ratings_cnt['log_count'] = np.log(df_ratings_cnt['count'])
df_ratings_cnt

count  log_count
0.0  15235120248  23.446869
0.5       442388  12.999943
1.0       886233  13.694735
1.5       441354  12.997603
2.0      1850627  14.431035
2.5      1373419  14.132814
3.0      5515668  15.523103
3.5      3404360  15.040568
4.0      7394710  15.816275
4.5      2373550  14.679897
5.0      4071135  15.219432

It's interesting that there are more people giving rating score of 3 and 4 than other scores 

#### 2. Plot rating frequency of all movies

In [ ]:
df_ratings.head()

userId  movieId  rating
0       1      307     3.5
1       1      481     3.5
2       1     1091     1.5
3       1     1257     4.5
4       1     1449     4.5

In [ ]:
# get rating frequency
df_movies_cnt = pd.DataFrame(df_ratings.groupby('movieId').size(), columns=['count'])
df_movies_cnt.head()

count
movieId       
1        68469
2        27143
3        15585
4         2989
5        15474

The distribution of ratings among movies often satisfies a property in real-world settings,
which is referred to as the long-tail property. According to this property, only a small
fraction of the items are rated frequently. Such items are referred to as popular items. The
vast majority of items are rated rarely. This results in a highly skewed distribution of the
underlying ratings.

Let's plot the same distribution but with log scale

We can see that roughly 10,000 out of 53,889 movies are rated more than 100 times. More interestingly, roughly 20,000 out of 53,889 movies are rated less than only 10 times. Let's look closer by displaying top quantiles of rating counts

In [ ]:
df_movies_cnt['count'].quantile(np.arange(1, 0.6, -0.05))

1.00    97999.0
0.95     1855.0
0.90      531.0
0.85      205.0
0.80       91.0
0.75       48.0
0.70       28.0
0.65       18.0
Name: count, dtype: float64

So about 1% of movies have roughly 97,999 or more ratings, 5% have 1,855 or more, and 20% have 100 or more. Since we have so many movies, we'll limit it to the top 25%. This is arbitrary threshold for popularity, but it gives us about 13,500 different movies. We still have pretty good amount of movies for modeling. There are two reasons why we want to filter to roughly 13,500 movies in our dataset.
 - Memory issue: we don't want to run into the “MemoryError” during model training
 - Improve KNN performance: lesser known movies have ratings from fewer viewers, making the pattern more noisy. Droping out less known movies can improve recommendation quality

In [ ]:
print(df_ratings.head())
print(df_movies.head())

   userId  movieId  rating
0       1      307     3.5
1       1      481     3.5
2       1     1091     1.5
3       1     1257     4.5
4       1     1449     4.5
   movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [ ]:
# filter data
popularity_thres = 50
popular_movies = list(set(df_movies_cnt.query('count >= @popularity_thres').index))
df_ratings = pd.merge(df_movies,df_ratings)
df_ratings_drop_movies = df_ratings[df_ratings.movieId.isin(popular_movies)]
print('shape of original ratings data: ', df_ratings.shape)
print('shape of ratings data after dropping unpopular movies: ', df_ratings_drop_movies.shape)

shape of original ratings data:  (27753444, 4)
shape of ratings data after dropping unpopular movies:  (27430748, 4)


After dropping 75% of movies in our dataset, we still have a very large dataset. So next we can filter users to further reduce the size of data

In [ ]:
# get number of ratings given by every user
df_users_cnt = pd.DataFrame(df_ratings_drop_movies.groupby('userId').size(), columns=['count'])
df_users_cnt.head()

count
userId       
1          16
2          15
3          11
4         736
5          72

In [ ]:
df_users_cnt['count'].quantile(np.arange(1, 0.5, -0.05))

1.00    9384.0
0.95     403.0
0.90     239.0
0.85     164.0
0.80     121.0
0.75      94.0
0.70      73.0
0.65      58.0
0.60      47.0
0.55      37.0
Name: count, dtype: float64

We can see that the distribution of ratings by users is very similar to the distribution of ratings among movies. They both have long-tail property. Only a very small fraction of users are very actively engaged with rating movies that they watched. Vast majority of users aren't interested in rating movies. So we can limit users to the top 40%, which is about 113,291 users.

In [ ]:
# filter data
ratings_thres = 50
active_users = list(set(df_users_cnt.query('count >= @ratings_thres').index))
df_ratings_drop_users = df_ratings_drop_movies[df_ratings_drop_movies.userId.isin(active_users)]
print('shape of original ratings data: ', df_ratings.shape)
print('shape of ratings data after dropping both unpopular movies and inactive users: ', df_ratings_drop_users.shape)

shape of original ratings data:  (27753444, 4)
shape of ratings data after dropping both unpopular movies and inactive users:  (24178982, 4)


## 3. Train KNN model for item-based collaborative filtering
 - Reshaping the Data
 - Fitting the Model

#### 1. Reshaping the Data
For K-Nearest Neighbors, we want the data to be in an (artist, user) array, where each row is a movie and each column is a different user. To reshape the dataframe, we'll pivot the dataframe to the wide format with movies as rows and users as columns. Then we'll fill the missing observations with 0s since we're going to be performing linear algebra operations (calculating distances between vectors). Finally, we transform the values of the dataframe into a scipy sparse matrix for more efficient calculations.

In [ ]:
df_ratings_drop_users

movieId             title  userId  rating
0               1  Toy Story (1995)       4     4.0
1               1  Toy Story (1995)      10     5.0
2               1  Toy Story (1995)      14     4.5
3               1  Toy Story (1995)      15     4.0
7               1  Toy Story (1995)      31     5.0
...           ...               ...     ...     ...
27752464   192307    The Meg (2018)  270572     3.0
27752465   192307    The Meg (2018)  272459     5.0
27752466   192307    The Meg (2018)  275841     1.5
27752467   192307    The Meg (2018)  279522     3.0
27752468   192307    The Meg (2018)  279713     3.0

[24178982 rows x 4 columns]

In [ ]:
moviemat = df_ratings_drop_users.pivot_table(index='userId', columns='title', values='rating')
moviemat

In [ ]:
user_rating = moviemat.dropna(thresh=10, axis=1).fillna(0,axis=1)
user_rating.head(100)

title   "Great Performances" Cats (1998)  ...  À nous la liberté (Freedom for Us) (1931)
userId                                    ...                                           
4                                    0.0  ...                                        0.0
5                                    0.0  ...                                        0.0
10                                   0.0  ...                                        0.0
14                                   0.0  ...                                        0.0
15                                   0.0  ...                                        0.0
...                                  ...  ...                                        ...
255                                  0.0  ...                                        0.0
256                                  0.0  ...                                        0.0
258                                  0.0  ...                                        0.0
268                                  0.0  ...                                        0.0
269                                  0.0  ...                                        0.0

[100 rows x 13357 columns]

In [ ]:
user_rating

title   "Great Performances" Cats (1998)  ...  À nous la liberté (Freedom for Us) (1931)
userId                                    ...                                           
4                                    0.0  ...                                        0.0
5                                    0.0  ...                                        0.0
10                                   0.0  ...                                        0.0
14                                   0.0  ...                                        0.0
15                                   0.0  ...                                        0.0
...                                  ...  ...                                        ...
283215                               0.0  ...                                        0.0
283219                               0.0  ...                                        0.0
283222                               0.0  ...                                        0.0
283224                               0.0  ...                                        0.0
283228                               0.0  ...                                        0.0

[109483 rows x 13357 columns]

In [ ]:
import pickle

In [ ]:
pickle.dump(user_rating, open("moviemat", 'wb'), protocol=4)

In [ ]:
corrMatrix = user_rating.corr(method='pearson')
corrMatrix.head(100)

In [ ]:
moviemat

title   "Great Performances" Cats (1998)  ...  À nous la liberté (Freedom for Us) (1931)
userId                                    ...                                           
4                                    NaN  ...                                        NaN
5                                    NaN  ...                                        NaN
10                                   NaN  ...                                        NaN
14                                   NaN  ...                                        NaN
15                                   NaN  ...                                        NaN
...                                  ...  ...                                        ...
283215                               NaN  ...                                        NaN
283219                               NaN  ...                                        NaN
283222                               NaN  ...                                        NaN
283224                               NaN  ...                                        NaN
283228                               NaN  ...                                        NaN

[109483 rows x 13357 columns]

In [ ]:
# import pickle
# pickle.dump(moviemat, open("moviemat", 'wb'), protocol=4)

In [ ]:
# # pivot and create movie-user matrix
# movie_user_mat = df_ratings_drop_users.pivot(index='userId', columns='title

# ', values='rating').fillna(0)
# # create mapper from movie title to index

# # transform matrix to scipy sparse matrix
# movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [ ]:
moviemat

title   "Great Performances" Cats (1998)  ...  À nous la liberté (Freedom for Us) (1931)
userId                                    ...                                           
4                                    NaN  ...                                        NaN
5                                    NaN  ...                                        NaN
10                                   NaN  ...                                        NaN
14                                   NaN  ...                                        NaN
15                                   NaN  ...                                        NaN
...                                  ...  ...                                        ...
283215                               NaN  ...                                        NaN
283219                               NaN  ...                                        NaN
283222                               NaN  ...                                        NaN
283224                               NaN  ...                                        NaN
283228                               NaN  ...                                        NaN

[109483 rows x 13357 columns]

In [ ]:
# user_movie_mat = movie_user_mat.T
# user_movie_mat

def standard(row):
    return (row - row.mean())/(row.max()-row.min())

ratings_stand = user_movie_mat.apply(standard).fillna(0)

ratings_stand

In [ ]:
# user_movie_mat

In [ ]:
# import pickle
# moviemat = pickle.load(open('drive/My Drive/moviemat','rb'))
# moviemat

In [ ]:
def standard(row):
    return (row - row.mean())/(row.max()-row.min())

ratings_stand = moviemat.apply(standard).fillna(0)

ratings_stand

title   "Great Performances" Cats (1998)  ...  À nous la liberté (Freedom for Us) (1931)
userId                                    ...                                           
4                                    0.0  ...                                        0.0
5                                    0.0  ...                                        0.0
10                                   0.0  ...                                        0.0
14                                   0.0  ...                                        0.0
15                                   0.0  ...                                        0.0
...                                  ...  ...                                        ...
283215                               0.0  ...                                        0.0
283219                               0.0  ...                                        0.0
283222                               0.0  ...                                        0.0
283224                               0.0  ...                                        0.0
283228                               0.0  ...                                        0.0

[109483 rows x 13357 columns]

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
user_rating = moviemat.dropna(thresh=10, axis=1).fillna(0,axis=1)
user_rating

In [ ]:
# ratings_stand.to_csv('Standardmat.csv')

In [ ]:
# ratings_stand.to_csv()

In [ ]:
import pickle

In [ ]:
pickle.dump(ratings_stand, open("standardized_moviemat", 'wb'), protocol=4)

In [ ]:
# print("Hello World")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(ratings_stand)

cosine_sim

In [ ]:
db = pickle.load(dbfile)

In [ ]:
import pickle

In [ ]:
pickle.dump(ratings_stand, open("rating_standars", 'wb'), protocol=4)

In [ ]:
mu_mat = pickle.load(open('drive/My Drive/movie_user_mat', 'rb'))
mu_mat

userId   4       5       10      14      ...  283219  283222  283224  283228
movieId                                  ...                                
1           4.0     0.0     5.0     4.5  ...     4.0     0.0     0.0     4.5
2           4.0     0.0     0.0     4.0  ...     0.0     0.0     0.0     0.0
3           0.0     0.0     0.0     0.0  ...     0.0     0.0     4.0     0.0
4           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
5           2.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
...         ...     ...     ...     ...  ...     ...     ...     ...     ...
191351      0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
191367      0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
191799      0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
192283      0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
192307      0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[13360 rows x 109483 columns]

In [ ]:
sm = pd.read_csv('/content/drive/My Drive/Standardmat.csv')

In [ ]:
um_mat = mu_mat

def standard(row):
    return (row - row.mean())/(row.max()-row.min())

ratings_stand = um_mat.apply(standard).fillna(0)

ratings_stand

userId     4         5        10      ...    283222    283224    283228
movieId                               ...                              
1        0.762567 -0.004596  0.99241  ... -0.003593 -0.018383  0.892657
2        0.762567 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
3       -0.037433 -0.004596 -0.00759  ... -0.003593  0.781617 -0.007343
4       -0.037433 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
5        0.362567 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
...           ...       ...      ...  ...       ...       ...       ...
191351  -0.037433 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
191367  -0.037433 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
191799  -0.037433 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
192283  -0.037433 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343
192307  -0.037433 -0.004596 -0.00759  ... -0.003593 -0.018383 -0.007343

[13360 rows x 109483 columns]

In [ ]:
pickle.dump(movie_user_mat, open("movie_user_mat", 'wb'), protocol=4)

In [ ]:
pickle.dump(movie_user_mat, open("movie_user_mat", 'wb'), protocol=4)

In [ ]:
pickle.dump(ratings_stand, open("user_movie_normallized", 'wb'), protocol=4)

In [ ]:
mat = df_movies.as_matrix()
mat

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[1, 'Toy Story (1995)'],
       [2, 'Jumanji (1995)'],
       [3, 'Grumpier Old Men (1995)'],
       ...,
       [193880, 'Her Name Was Mumu (2016)'],
       [193882, 'Flora (2017)'],
       [193886, 'Leal (2018)']], dtype=object)

In [ ]:
lst = []
idx=0
for i in user_movie_mat:
  if idx==5800:break
  i = mat[i][1]
  idx=idx+1


#### 2. Fitting the Model
Time to implement the model. We'll initialize the NearestNeighbors class as model_knn and fit our sparse matrix to the instance. By specifying the metric = cosine, the model will measure similarity bectween artist vectors by using cosine similarity.

In [ ]:
movie_user_mat_sparse = csr_matrix(mu_mat.values)

In [ ]:
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(df_movies.set_index('movieId').loc[mu_mat.index].title))
 }

In [ ]:
#%env JOBLIB_TEMP_FOLDER=/tmp
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [ ]:
import pickle
with open('model_knn_pikle','wb') as f:
  pickle.dump(model_knn,f)

In [ ]:
with open('mapper_pikle.pkl','wb') as f:
  pickle.dump(movie_to_idx,f)

In [ ]:
with open('sparse_mat_pickle','wb') as f:
  pickle.dump(movie_user_mat_sparse,f)

## 4. Use this trained model to make movie recommendations to myself
And we're finally ready to make some recommendations!

In [ ]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]
    print(match_tuple[0][1])



def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: movie-user matrix

    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    #model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_movie)
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [ ]:
my_favorite = 'iron man'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=20)

You have input movie: iron man
Found possible matches in our database: ['Iron Man (2008)', 'Iron Man 3 (2013)', 'Iron Man 2 (2010)']

Recommendation system start to make inference
......

Recommendations for iron man:
1: Prestige, The (2006), with distance of 0.4582262635231018
2: X-Men: First Class (2011), with distance of 0.4536136984825134
3: District 9 (2009), with distance of 0.4504380226135254
4: 300 (2007), with distance of 0.44966357946395874
5: Guardians of the Galaxy (2014), with distance of 0.4419601559638977
6: Lord of the Rings: The Return of the King, The (2003), with distance of 0.4389902949333191
7: V for Vendetta (2006), with distance of 0.43672531843185425
8: Up (2009), with distance of 0.43219220638275146
9: Dark Knight Rises, The (2012), with distance of 0.4312893748283386
10: Casino Royale (2006), with distance of 0.42907726764678955
11: Bourne Ultimatum, The (2007), with distance of 0.4217848777770996
12: Sherlock Holmes (2009), with distance of 0.4190899133682251

In [ ]:
my_favorite = 'justice league'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=20)

You have input movie: justice league
Found possible matches in our database: ['Justice League (2017)', 'Justice League Dark (2017)', 'Justice League: War (2014)', 'Justice League: Doom (2012) ']

Recommendation system start to make inference
......

Recommendations for justice league:
1: The Hitman's Bodyguard (2017), with distance of 0.6561742424964905
2: Doctor Strange (2016), with distance of 0.6550297737121582
3: Logan (2017), with distance of 0.6542268991470337
4: Valerian and the City of a Thousand Planets (2017), with distance of 0.6523512005805969
5: Ready Player One, with distance of 0.6509034633636475
6: X-Men: Apocalypse (2016), with distance of 0.6486040949821472
7: War for the Planet of the Apes (2017), with distance of 0.6464254856109619
8: Deadpool 2 (2018), with distance of 0.6400861740112305
9: Guardians of the Galaxy 2 (2017), with distance of 0.6343815326690674
10: Bright (2017), with distance of 0.6339102983474731
11: Suicide Squad (2016), with distance of 0.6279527

In [ ]:
my_favorite = '3 idiots'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=20)

You have input movie: 3 idiots
Found possible matches in our database: ['3 Idiots (2009)']

Recommendation system start to make inference
......

Recommendations for 3 idiots:
1: Bhaag Milka Bhaag (2013), with distance of 0.7919452786445618
2: Sholay (1975), with distance of 0.7891584038734436
3: Barfi! (2012), with distance of 0.7831109166145325
4: Dilwale Dulhania Le Jayenge (1995), with distance of 0.781316876411438
5: Gangs of Wasseypur (2012), with distance of 0.7808258533477783
6: OMG Oh My God! (2012), with distance of 0.7763568758964539
7: Kal Ho Naa Ho (2003), with distance of 0.7711299657821655
8: Wednesday!, A (2008), with distance of 0.7671446204185486
9: Dangal (2016), with distance of 0.7581744194030762
10: Kahaani (2012), with distance of 0.7578554153442383
11: Ghajini (2008), with distance of 0.7530505657196045
12: Jab We Met (2007), with distance of 0.7524836659431458
13: My Name is Khan (2010), with distance of 0.7508781552314758
14: D.C.H. (Dil Chahta Hai) (2001), wi

In [ ]:
my_favorite = 'dilwale dulhania le jayenge'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=20)

You have input movie: dilwale dulhania le jayenge
Found possible matches in our database: ['Dilwale Dulhania Le Jayenge (1995)']

Recommendation system start to make inference
......

Recommendations for dilwale dulhania le jayenge:
1: Ghajini (2008), with distance of 0.6336442232131958
2: Don (1978), with distance of 0.628887414932251
3: Jodhaa Akbar (2008), with distance of 0.6235666275024414
4: Devdas (2002), with distance of 0.6181485652923584
5: Om Shanti Om (2007), with distance of 0.6118590235710144
6: Swades: We, the People (Our Country) (2004), with distance of 0.6103717088699341
7: Zindagi Na Milegi Dobara (2011), with distance of 0.6100454330444336
8: Yeh Jawaani Hai Deewani (2013), with distance of 0.6099769473075867
9: Raajneeti (2010), with distance of 0.6027723550796509
10: Sometimes Happiness, Sometimes Sorrow (Kabhi Khushi Kabhie Gham) (2001), with distance of 0.6011155843734741
11: Chak De India! (2007), with distance of 0.5955387353897095
12: Sholay (1975), with dist

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


This is very interesting that my **KNN** model recommends movies that were also produced in very similar years. However, the cosine distance of all those recommendations are actually quite small. This is probabily because there is too many zero values in our movie-user matrix. With too many zero values in our data, the data sparsity becomes a real issue for **KNN** model and the distance in **KNN** model starts to fall apart. So I'd like to dig deeper and look closer inside our data.

#### (extra inspection) 
Let's now look at how sparse the movie-user matrix is by calculating percentage of zero values in the data.

In [ ]:
# calcuate total number of entries in the movie-user matrix
num_entries = movie_user_mat.shape[0] * movie_user_mat.shape[1]
# calculate total number of entries with zero values
num_zeros = (movie_user_mat==0).sum(axis=1).sum()
# calculate ratio of number of zeros to number of entries
ratio_zeros = num_zeros / num_entries
print('There is about {:.2%} of ratings in our data is missing'.format(ratio_zeros))

This result confirms my hypothesis. The vast majority of entries in our data is zero. This explains why the distance between similar items or opposite items are both pretty large.

## 5. Deep dive into the bottleneck of item-based collaborative filtering.
 - cold start problem
 - data sparsity problem
 - popular bias (how to recommend products from the tail of product distribution)
 - scalability bottleneck

We saw there is 98.35% of user-movie interactions are not yet recorded, even after I filtered out less-known movies and inactive users. Apparently, we don't even have sufficient information for the system to make reliable inferences for users or items. This is called **Cold Start** problem in recommender system.

There are three cases of cold start:

1. New community: refers to the start-up of the recommender, when, although a catalogue of items might exist, almost no users are present and the lack of user interaction makes very hard to provide reliable recommendations
2. New item: a new item is added to the system, it might have some content information but no interactions are present
3. New user: a new user registers and has not provided any interaction yet, therefore it is not possible to provide personalized recommendations

We are not concerned with the last one because we can use item-based filtering to make recommendations for new user. In our case, we are more concerned with the first two cases, especially the second case.

The item cold-start problem refers to when items added to the catalogue have either none or very little interactions. This constitutes a problem mainly for collaborative filtering algorithms due to the fact that they rely on the item's interactions to make recommendations. If no interactions are available then a pure collaborative algorithm cannot recommend the item. In case only a few interactions are available, although a collaborative algorithm will be able to recommend it, the quality of those recommendations will be poor. This arises another issue, which is not anymore related to new items, but rather to unpopular items. In some cases (e.g. movie recommendations) it might happen that a handful of items receive an extremely high number of iteractions, while most of the items only receive a fraction of them. This is also referred to as popularity bias. Please recall previous long-tail skewed distribution of movie rating frequency plot.

In addtition to that, scalability is also a big issue in KNN model too. Its time complexity is O(nd + kn), where n is the cardinality of the training set and d the dimension of each sample. And KNN takes more time in making inference than training, which increase the prediction latency

## 6. Further study

Use spark's ALS to solve above problems